In [ ]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import netCDF4
import fill_values as fv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plt_ce_image as im
from matplotlib.gridspec import GridSpec
import datetime
import calc_date as cd
import save_fig as sF
import createFolder as cF

In [ ]:
year = '2016'
month = '12'
#t = ['22']
t = ['20', '21', '22', '23', '24', '25', '26', '27']
tid = '00'
station = 'Haukeliseter'

In [ ]:
### Figures 
savefig = 0
figdir = '../../Figures/Retrieval_MEPS/Fraction'
cF.createFolder('%s/' %(figdir))
form = 'png'

In [ ]:
################################################################
### MEPS
level = 'sfc'
nc_dir_sfc = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
level = 'ml'
nc_dir_ml = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
# snow fall amount
var_name_snow = 'snowfall_amount_ml'
snow_dir = '%s/%s' %(nc_dir_ml,var_name_snow)
# graupel fall amount
var_name_graupel = 'graupelfall_amount_ml'
graupel_dir = '%s/%s' %(nc_dir_ml,var_name_graupel)
# atmosphere_cloud_ice_content_ml
var_name_cl_ice = 'atmosphere_cloud_ice_content_ml'
cl_ice_dir = '%s/%s' %(nc_dir_ml,var_name_cl_ice)
## atmosphere cloud ice content
#var_name_ice_cont = 'atmosphere_cloud_ice_content_ml'
#ice_cont_dir = '%s/%s' %(nc_dir_ml,var_name_ice_cont)
################################################################
### Retrieval 
nc_dir_retrieval = '../../Data/Retrieved_SWC'

################################################################

In [ ]:
snow_file    = dict()
graupel_file = dict()
cl_ice_file  = dict()

time_ml      = dict()

fn_snow           = dict()
fn_graupel        = dict()
fn_cl_ice         = dict()

snow_amount_ml    = dict()
graupel_amount_ml = dict()
cl_ice_ml         = dict()

thickness_ml      = dict()
ice_amount_ml     = dict()
height_ml            = dict()

In [ ]:
for day in t:
## RETRIEVAL ######
    Retrieval_file = '%s/out_swc_b6_%s%s%s.nc' %(nc_dir_retrieval, year, month, day)
    fn = netCDF4.Dataset(Retrieval_file)
    retrieved_snowfall_amount = fn.variables['retrieved_snowfall_amount'][:]
    SWC = fn.variables['SWC'][:]
## MEPS ######
    for ens_memb in range(0,10):
        snow_file[ens_memb]    = '%s/%s%s%s_%s_%s.nc' %(snow_dir,year,month,day,tid,ens_memb)
        graupel_file[ens_memb] = '%s/%s%s%s_%s_%s.nc' %(graupel_dir,year,month,day,tid,ens_memb)
        cl_ice_file[ens_memb]  = '%s/%s%s%s_%s_%s.nc' %(cl_ice_dir,year,month,day,tid,ens_memb)
    
        fn_snow[ens_memb]      = netCDF4.Dataset(snow_file[ens_memb])
        fn_graupel[ens_memb]   = netCDF4.Dataset(graupel_file[ens_memb]) 
        fn_cl_ice[ens_memb]    = netCDF4.Dataset(cl_ice_file[ens_memb])
        
# Read in time to know initialisation time (plotting)    
        time_ml[ens_memb] = fn_snow[ens_memb].variables['time']
        time_ml[ens_memb] = fv.fill_nan(time_ml[ens_memb][:])
        
        ini_day = datetime.datetime.utcfromtimestamp(time_ml[0][0]).day   # day of initialisation
        hh = datetime.datetime.utcfromtimestamp(time_ml[0][0]).hour       # first hour of initialisation?
        calday, calmon = cd.get_dayname(year, month, ini_day)
        
# Read in the variable name which should be plotted  
        snow_amount_ml[ens_memb]    = fn_snow[ens_memb].variables[var_name_snow]
        graupel_amount_ml[ens_memb] = fn_graupel[ens_memb].variables[var_name_graupel]
        cl_ice_ml[ens_memb]         = fn_cl_ice[ens_memb].variables[var_name_cl_ice]


    
# substitute missing values with nan       
        snow_amount_ml[ens_memb]    = fv.fill_nan(snow_amount_ml[ens_memb][:])
        graupel_amount_ml[ens_memb] = fv.fill_nan(graupel_amount_ml[ens_memb][:])
        cl_ice_ml[ens_memb]         = fv.fill_nan(cl_ice_ml[ens_memb][:])
# get ice amount (snow+graupel)
        ice_amount_ml[ens_memb]     = snow_amount_ml[ens_memb] + graupel_amount_ml[ens_memb] + cl_ice_ml[ens_memb]
    
# layer thickness to calculate height
        thickness_ml[ens_memb]      = fn_snow[ens_memb].variables['layer_thickness']
        thickness_ml[ens_memb]      = fv.fill_nan(thickness_ml[ens_memb][:])
    
        thickness = thickness_ml[ens_memb]
        h_above = []
        h_above.append(np.zeros(thickness[:,:].shape[0]))
        th_arr = np.nansum([np.asarray(h_above)[:,0], thickness[:,0]], axis = 0)
        h_above.append(th_arr)
        for i in range(2,thickness[:,:].shape[1]):
            th_arr = np.nansum([h_above[i-1], thickness[:,i-1]], axis = 0)
            h_above.append(th_arr)
        height_ml[ens_memb] = np.transpose(h_above)


    
# get only valuable values not nan 
        time_ml[ens_memb]       = (time_ml[ens_memb][~np.isnan(time_ml[ens_memb])])
        height_ml[ens_memb]        = (height_ml[ens_memb][~np.isnan(ice_amount_ml[ens_memb]).any(axis=1),:])
        thickness_ml[ens_memb]  = (thickness_ml[ens_memb][~np.isnan(ice_amount_ml[ens_memb]).any(axis=1),:])  
        ice_amount_ml[ens_memb] = (ice_amount_ml[ens_memb][~np.isnan(ice_amount_ml[ens_memb]).any(axis=1)])
    
# convert the snow amount from [kg/m^2] into [g/m^2] by multiply with 1000
        ice_amount_ml[ens_memb] = (ice_amount_ml[ens_memb]*1000.)

# calculate mean of snowfall amount in each layer, every 200m

    nlay= 14
    bot = []
    top = []
    for k in range(0, nlay):
        bot.append(300.+ k*200)
        top.append(bot[k]+200)
#bot, top


    avg_SWC_ml = dict()
    for ens_memb in range(0,10):
        SWC_ml = []
#avg_height_ml = []
        h_mid = []
        for k in range(0, nlay):
            tidx,hidx = np.where(np.logical_and(height_ml[ens_memb][:24,:] >= bot[k], height_ml[ens_memb][:24,:] < top[k]))
    
    # average in layer
            SWC_ml.append(np.nanmean(ice_amount_ml[ens_memb][:24,hidx[:]], axis = 1))
            h_mid.append((bot[k]+top[k])/2)
        avg_SWC_ml[ens_memb] = SWC_ml

    
#pd.DataFrame(avg_SWC_ml[0])
# average over each hour, take 30min before and after full hour
    avg_SWC_ret = []
    avg_snow_amount = []
    for i in range(30,int(SWC.shape[1]),60):
        avg_SWC_ret.append(np.nanmean(SWC[:,i:i+59],axis = 1))
        avg_snow_amount.append(np.nanmean(retrieved_snowfall_amount[:,i:i+59],axis =1 ))

    avg_SWC_ret = (np.transpose(avg_SWC_ret))
    avg_snow_amount = np.transpose(avg_snow_amount)

#pd.DataFrame(avg_SWC_ret)

# calculate the fraction of the model/retreival
    SWC_frac = dict()
    for ens_memb in range(0,10):
        if (np.asarray(avg_SWC_ml[ens_memb]).shape == (avg_SWC_ret).shape):
            SWC_frac[ens_memb] = avg_SWC_ml[ens_memb]/avg_SWC_ret
        else:
            SWC_frac[ens_memb] = np.empty(shape=avg_SWC_ret.shape)
            SWC_frac[ens_memb][:] = np.nan

#SWC_frac = avg_SWC_ml/avg_SWC_ret
#a,b = np.where(avg_snow_amount[:,:] < 0.01)
#for ens_memb in range(0,10):
 #   SWC_frac[ens_memb][a[:],b[:]] = np.nan
#np.asarray(avg_SWC_ml)[a[:],b[:]] = np.nan
#avg_SWC_ret[a[:],b[:]]= np.nan

#pd.DataFrame(SWC_frac[0])
#################################################
#### PLOTTING ######
    fig_name = '%s%s%s.%s' %(year,month,ini_day,form)
    fig = plt.figure(figsize=(18.,10))
    gs = GridSpec(1,2)
    ax0 = plt.subplot(gs[0,:])
    levels = np.arange(0, 1.01,0.01 )
#im0 = ax0.contourf(time_ml[0][:24],h_mid, SWC_frac,levels, cmap = 'seismic',
 #                 extend = 'max', alpha = 1.0, 
  #                     vmin = 0, vmax = 1, origin = 'lower')
    im0 = ax0.contourf(np.arange(0.5,24),h_mid, SWC_frac[0],levels, cmap = 'seismic',extend = 'max', alpha = 1.0, 
                       vmin = 0, vmax = 1., origin = 'lower')
    ax0.axis([0., 24., 0.,np.max(h_mid)])
    im.labels_x(ax0, xticks =im.xticks2)
    im.labels_y(ax0, yticks=im.yticks1, yl = im.yl1, label_txt = 'height [km]')
    im.add_colorbar(fig,im0,ax0,levels[::10],'SWC$_{MEPS}$(EM0)/SWC$_{RETRIVAL}$')

    ax0.text(0.02,0.95, '%s, %s %s %s' %(calday, day, calmon, year),
            verticalalignment ='top', horizontalalignment='left',
             transform = ax0.transAxes,
             color = im.date_blue, fontsize = im.fontsize,
             bbox={'facecolor':'white','alpha':1.,'pad':10})
    if savefig == 1:
        sF.save_figure_portrait(figdir, fig_name, form)
        print('saved: %s/%s' %(figdir, fig_name))
    else:
        plt.show()
    plt.close()
    
    
    for ens_memb in range(0,10):
        fn_snow[ens_memb].close()
        fn_graupel[ens_memb].close()
        fn_cl_ice[ens_memb].close()